In [3]:
# !pip install git+https://github.com/huggingface/transformers -qqq
# !pip install datasets -qqq
# !pip install bitsandbytes -qqq
# !pip install huggingface_hub -qqq
# !pip install peft -qqq
# !pip install accelerate -qqq
# !pip install bitsandbytes -qqq
# !pip install trl -qqq
# !pip install wandb -qqq
# !pip install ipywidgets -qqq
# !pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 20.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [7]:
# login outside of jupyter notebook

# !huggingface-cli whoami

# import wandb
# wandb.login()

ogbrandt


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1", 
    padding_side="right",
    add_eos_token=True,
    add_bos_token=True,
                                          )
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


inf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

device = "cuda"
model_inputs = encodeds.to(device)

generated_ids = inf_model.generate(model_inputs, max_new_tokens=256, pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Loading checkpoint shards: 100%|██████████| 2/2 [01:47<00:00, 53.86s/it]

No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



<s> [INST] What is your favourite condiment? [/INST]

I'm not sure if this is a condiment, but I love ketchup. I'm not sure why, but I love it. I'm not a big fan of mustard, but I do like it on hot dogs. I'm not a big fan of mayonnaise, but I do like it on sandwiches. I'm not a big fan of relish, but I do like it on hot dogs. I'm not a big fan of pickles, but I do like them on sandwiches. I'm not a big fan of olives, but I do like them on pizza. I'm not a big fan of cheese, but I do like it on pizza. I'm not a big fan of tomatoes, but I do like them on pizza. I'm not a big fan of onions, but I do like them on pizza. I'm not a big fan of peppers, but I do like them on pizza. I'm not a big fan of mushrooms, but I do like them on pizza. I'm not a big fan of sausage, but I do like it on pizza. I'm not a big fan of pepperoni, but I do like it on pizza. I'm not a big fan of anchovies, but I do like them on pizza. I'm not a big fan of bacon, but I do like it on pizza. I'm not a big fan of ham

In [14]:
# Empty VRAM
del model_inputs
del inf_model

import gc
gc.collect()
gc.collect()

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

4672979968

In [21]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Fri Jan 19 14:00:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   38C    P0             190W / 300W |  15211MiB / 23028MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# attach our trained QLora adapters to base model
base_model_name = "mistralai/Mistral-7B-v0.1"
# base_model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# You can just use model.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

wandb: Currently logged in as: ongjjbrandon. Use `wandb login --relogin` to force relogin


True

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ongjjbrandon/mistral-pjf-ft-v1/model-3uzptk49:v0', type='model')
artifact_dir = artifact.download()
run.finish()

In [ ]:
model = PeftModel.from_pretrained(inf_model, "/content/artifacts/model-3uzptk49:v0")

In [ ]:
prompt = "What is a neural network??"

device = "cuda" if torch.cuda.is_available() else "cpu"
model_input = tokenizer(prompt, return_tensors="pt").to(device)

_ = model.eval()
with torch.no_grad():
    out = model.generate(**model_input, max_new_tokens=100)

print(tokenizer.decode(out[0], skip_special_tokens=True))

In [20]:
test_set = ["how can I jump higher","how to dunk?","are barefoot shoots actually beneficial?","what are the most important factors to becoming an elite athlete?",
"what are the core principles behind nutrition?",
"how do i lose weight while gaining muscle",
"how do i get more energy?",
"are plant based diets beneficial for better health?",
"what are some quick wins to becoming a better athlete?",
"does contrast therapy (ice bath and hot sauna) actually work?",
"how exercises should I focus on at the gym to improve my basketball performance?"]

for prompt in test_set:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(device)
    model_input = tokenizer(prompt, return_tensors="pt").to(device)
    
    _ = inf_model.eval()
    with torch.no_grad():
      out = inf_model.generate(**model_input, max_new_tokens=216)
    
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    print()

cuda


NameError: name 'inf_model' is not defined